# Problem extension 1: (yellow line extension)


In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import *
import numpy as np

In [2]:
# Import demand data
demand_yellow = pd.read_excel('Demand_estimation_Index.xlsx', sheet_name='yellow', index_col = 0)
new_demand_yellow = demand_yellow * 1.12 / 0.46 
# Demand * 1.12 because there will be 12% increase
# And it will / 0.46 because it will be the demand before covid
# The same apply to orange line extension

# Change to array
arr_demand_yellow = new_demand_yellow.to_numpy()

# Length
num_hour_period = demand_yellow.shape[0] # i = 20
num_day_type = demand_yellow.shape[1] # j = 4

new_demand_yellow

,Weekday,Friday,Saturday,Sunday
1,2220.521739,2220.521739,2220.521739,2220.521739
2,4996.173913,4996.173913,2220.521739,2220.521739
3,4996.173913,4996.173913,3330.782609,3330.782609
4,4163.478261,4163.478261,4163.478261,4163.478261
5,3330.782609,3330.782609,4996.173913,4996.173913
6,3330.782609,3330.782609,3330.782609,3330.782609
7,3330.782609,3330.782609,3330.782609,3330.782609
8,3330.782609,3330.782609,3330.782609,3330.782609
9,3330.782609,3330.782609,3330.782609,3330.782609
10,3330.782609,3330.782609,3330.782609,3330.782609


In [3]:
# Import capacity of each car
mr73_car = 160
mpm10_car = 172

mr73_seats_car = 40
mpm10_seats_car = 32

# Number of cars per train
yellow_num_cars = 9

# Import maximum average waiting time
yellow_mon_fri_nonpeak = 10
yellow_sat_sun = 10

# Service interval between 2 trains during peak periods
yellow_new_peak = 4

# Yellow line variable cost
yellow_variable_cost = 180 # yellow line -- weighted average variable cost per departure per line

# Define name
hour = ['5-6am', '6-7am', '7-8am', '8-9am', '9-10am', '10-11am', '11am-12pm', 
        '12-13pm', '13-14pm', '14-15pm', '15-16pm', '16-17pm', '17-18pm', '18-19pm', 
       '19-20pm', '20-21pm', '21-22pm', '22-23pm', '23pm-12am', '12am-1am']
time = ['Mon-Thu', 'Fri', 'Sat', 'Sun']

# Adding additional number of cars
yellow_additional_car = 108
yellow_additional_train = 12

In [4]:
# Create model
model_y = gp.Model('STM yellow line future plan minimize cost model')

Academic license - for non-commercial use only - expires 2022-09-25
Using license file /Users/AngelaChen/gurobi.lic


In [5]:
# Create decision variables
x_yellow = model_y.addVars(num_hour_period, num_day_type, vtype = GRB.INTEGER, 
                         name = ['Yellow #departure ' + str(i) + '_' + str(j) 
                               for i in hour for j in time])

In [6]:
# Yellow
obj = 4 * sum(x_yellow[i, 0] * yellow_variable_cost 
               for i in range(num_hour_period)) + sum(x_yellow[i, j] * yellow_variable_cost 
                                                      for i in range(num_hour_period) 
                                                      for j in range(1, num_day_type))

# Set objective function
model_y.setObjective(obj, GRB.MINIMIZE)

In [7]:
# Constraints
# 1. Capacity > Demand
for i in range(num_hour_period):
    for j in range(num_day_type):
        model_y.addConstr(x_yellow[i, j] * mr73_car * yellow_num_cars >= arr_demand_yellow[i, j])

In [8]:
# 2. Passenger waiting time < Maximum waiting time
peak = [2, 3, 11, 12]
non_peak = []
for num in range(20):
    if num not in peak:
        non_peak.append(num)

for i in peak:
    model_y.addConstr(x_yellow[i, 0] * yellow_new_peak >= 60)
    model_y.addConstr(x_yellow[i, 1] * yellow_new_peak >= 60)
    
for i in non_peak:
    model_y.addConstr(x_yellow[i, 0] * yellow_mon_fri_nonpeak >= 60)
    model_y.addConstr(x_yellow[i, 1] * yellow_mon_fri_nonpeak >= 60)

for i in range(num_hour_period):
    for j in range(2,4):
        model_y.addConstr(x_yellow[i, j] * yellow_sat_sun >= 60)

In [9]:
model_y.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 160 rows, 80 columns and 160 nonzeros
Model fingerprint: 0xb62edb32
Variable types: 0 continuous, 80 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e+00, 1e+03]
  Objective range  [2e+02, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 5e+03]
Found heuristic solution: objective 183600.00000
Presolve removed 160 rows and 80 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 183600 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.836000000000e+05, best bound 1.836000000000e+05, gap 0.0000%


In [10]:
model_y.objVal

183600.0

In [11]:
for var in model_y.getVars():
    if var.x > 0:
        print(var.varName, '=', round(var.x, 2))

Yellow #departure 5-6am_Mon-Thu = 6.0
Yellow #departure 5-6am_Fri = 6.0
Yellow #departure 5-6am_Sat = 6.0
Yellow #departure 5-6am_Sun = 6.0
Yellow #departure 6-7am_Mon-Thu = 6.0
Yellow #departure 6-7am_Fri = 6.0
Yellow #departure 6-7am_Sat = 6.0
Yellow #departure 6-7am_Sun = 6.0
Yellow #departure 7-8am_Mon-Thu = 15.0
Yellow #departure 7-8am_Fri = 15.0
Yellow #departure 7-8am_Sat = 6.0
Yellow #departure 7-8am_Sun = 6.0
Yellow #departure 8-9am_Mon-Thu = 15.0
Yellow #departure 8-9am_Fri = 15.0
Yellow #departure 8-9am_Sat = 6.0
Yellow #departure 8-9am_Sun = 6.0
Yellow #departure 9-10am_Mon-Thu = 6.0
Yellow #departure 9-10am_Fri = 6.0
Yellow #departure 9-10am_Sat = 6.0
Yellow #departure 9-10am_Sun = 6.0
Yellow #departure 10-11am_Mon-Thu = 6.0
Yellow #departure 10-11am_Fri = 6.0
Yellow #departure 10-11am_Sat = 6.0
Yellow #departure 10-11am_Sun = 6.0
Yellow #departure 11am-12pm_Mon-Thu = 6.0
Yellow #departure 11am-12pm_Fri = 6.0
Yellow #departure 11am-12pm_Sat = 6.0
Yellow #departure 11am-12p

# Problem extension 2: (orange line extension)

In [12]:
# Import demand data
demand_orange = pd.read_excel('Demand_estimation_Index.xlsx', sheet_name='orange', index_col = 0)
new_demand_orange = demand_orange * 1.12 / 0.46 # Post-covid, and with additional demand
arr_demand_orange = new_demand_orange.to_numpy()

In [13]:
# Number of cars per train
orange_num_cars = 9

# Import maximum average waiting time
orange_mon_fri_nonpeak = 10
orange_sat_sun = 12

# Service interval between 2 trains during peak periods
orange_new_peak = 2.5

# Yellow line variable cost
orange_variable_cost = 1519 # orange line -- weighted average variable cost per departure per line

In [14]:
# Create model
model_o = gp.Model('STM orange line future plan minimize cost model')

In [15]:
# Create decision variables
x_orange = model_o.addVars(num_hour_period, num_day_type, vtype = GRB.INTEGER, 
                         name = ['Orange #departure ' + str(i) + '_' + str(j) 
                               for i in hour for j in time])

In [16]:
# Yellow
obj = 4 * sum(x_orange[i, 0] * orange_variable_cost 
               for i in range(num_hour_period)) + sum(x_orange[i, j] * orange_variable_cost 
                                                      for i in range(num_hour_period) 
                                                      for j in range(1, num_day_type))

# Set objective function
model_o.setObjective(obj, GRB.MINIMIZE)

In [17]:
# Constraints
# 1. Capacity > Demand
for i in range(num_hour_period):
    for j in range(num_day_type):
        model_o.addConstr(x_orange[i, j] * mr73_car * orange_num_cars >= arr_demand_orange[i, j])

In [18]:
# 2. Passenger waiting time < Maximum waiting time
peak = [2, 3, 11, 12]
non_peak = []
for num in range(20):
    if num not in peak:
        non_peak.append(num)

for i in peak:
    model_o.addConstr(x_orange[i, 0] * orange_new_peak >= 60)
    model_o.addConstr(x_orange[i, 1] * orange_new_peak >= 60)
    
for i in non_peak:
    model_o.addConstr(x_orange[i, 0] * orange_mon_fri_nonpeak >= 60)
    model_o.addConstr(x_orange[i, 1] * orange_mon_fri_nonpeak >= 60)

for i in range(num_hour_period):
    for j in range(2,4):
        model_o.addConstr(x_orange[i, j] * orange_sat_sun >= 60)

In [19]:
model_o.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 160 rows, 80 columns and 160 nonzeros
Model fingerprint: 0x02014e52
Variable types: 0 continuous, 80 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 1e+03]
  Objective range  [2e+03, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 5e+04]
Found heuristic solution: objective 4877509.0000
Presolve removed 160 rows and 80 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 4.87751e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.877509000000e+06, best bound 4.877509000000e+06, gap 0.0000%


In [20]:
for var in model_o.getVars():
    if var.x > 0:
        print(var.varName, '=', round(var.x, 2))

Orange #departure 5-6am_Mon-Thu = 16.0
Orange #departure 5-6am_Fri = 16.0
Orange #departure 5-6am_Sat = 16.0
Orange #departure 5-6am_Sun = 16.0
Orange #departure 6-7am_Mon-Thu = 35.0
Orange #departure 6-7am_Fri = 35.0
Orange #departure 6-7am_Sat = 16.0
Orange #departure 6-7am_Sun = 16.0
Orange #departure 7-8am_Mon-Thu = 35.0
Orange #departure 7-8am_Fri = 35.0
Orange #departure 7-8am_Sat = 24.0
Orange #departure 7-8am_Sun = 24.0
Orange #departure 8-9am_Mon-Thu = 29.0
Orange #departure 8-9am_Fri = 29.0
Orange #departure 8-9am_Sat = 29.0
Orange #departure 8-9am_Sun = 29.0
Orange #departure 9-10am_Mon-Thu = 24.0
Orange #departure 9-10am_Fri = 24.0
Orange #departure 9-10am_Sat = 35.0
Orange #departure 9-10am_Sun = 35.0
Orange #departure 10-11am_Mon-Thu = 24.0
Orange #departure 10-11am_Fri = 24.0
Orange #departure 10-11am_Sat = 24.0
Orange #departure 10-11am_Sun = 24.0
Orange #departure 11am-12pm_Mon-Thu = 24.0
Orange #departure 11am-12pm_Fri = 24.0
Orange #departure 11am-12pm_Sat = 24.0
Ora

In [21]:
model_o.objVal

4877509.0